In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import threading
import time

class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)
	
	def nextValidId(self, orderId: int):
		super().nextValidId(orderId)
		self.nextorderId = orderId
		print('The next valid order id is: ', self.nextorderId)

	def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
		print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
	
	def openOrder(self, orderId, contract, order, orderState):
		print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

	def execDetails(self, reqId, contract, execution):
		print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)


def run_loop():
	app.run()

contract = Contract()
contract.symbol = "AAPL" # apple
contract.secType = "STK" # stk stand for stock
contract.currency = "USD" 
contract.exchange = "SMART"

app = IBapi()
app.connect('127.0.0.1', 7496, 123)

app.nextorderId = None

# print("Next Order ID: ", app.nextValidId(0))

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Check if the API is connected via orderid
while True:
	if isinstance(app.nextorderId, int):
		print('connected')
		print()
		break
	else:
		print('waiting for connection')
		time.sleep(5)

#Create order object
order = Order()
order.action = 'SELL'
order.totalQuantity = 2
order.orderType = 'LMT'
order.lmtPrice = 194.26
order.eTradeOnly = False
order.firmQuoteOnly = False

#Place order
app.placeOrder(app.nextorderId, contract, order)
#app.nextorderId += 1

time.sleep(30)

#Cancel order 
print('cancelling order')
app.cancelOrder(app.nextorderId)

time.sleep(3)
app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:cashhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


waiting for connection
The next valid order id is:  7
connected

openOrder id: 7 AAPL STK @ SMART : SELL LMT 2.0 Submitted
orderStatus - orderid: 7 status: Submitted filled 0.0 remaining 2.0 lastFillPrice 0.0
openOrder id: 7 AAPL STK @ SMART : SELL LMT 2.0 Submitted
orderStatus - orderid: 7 status: Submitted filled 0.0 remaining 2.0 lastFillPrice 0.0
Order Executed:  -1 AAPL STK USD 0000db5b.64c1154d.01.01 7 2.0 2
openOrder id: 7 AAPL STK @ SMART : SELL LMT 2.0 Filled
orderStatus - orderid: 7 status: Filled filled 2.0 remaining 0.0 lastFillPrice 194.26
openOrder id: 7 AAPL STK @ SMART : SELL LMT 2.0 Filled
orderStatus - orderid: 7 status: Filled filled 2.0 remaining 0.0 lastFillPrice 194.26


ERROR 7 10148 OrderId 7 that needs to be cancelled cannot be cancelled, state: Filled.


cancelling order
